# Regression Model B: Field 3

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 3 (F3)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E3" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E3" ),]
dE4=d4[which(d4$Estate=="E3" ),]
dE5=d5[which(d5$Estate=="E3" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,manuring.time,manuring.cost,prun.time,prun.cost,
14,1,1,1,1,1,1,1,1,1,1,1,0
13,1,1,1,1,1,1,1,1,1,0,0,2
8,1,1,1,1,1,1,1,0,0,1,1,2
9,1,1,1,1,1,1,1,0,0,0,0,4
,0,0,0,0,0,0,0,17,17,22,22,78


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost
  3   3  prun.time  prun.cost  manuring.time  manuring.cost
  3   4  prun.time  prun.cost  manuring.time  manuring.cost
  3   5  prun.time  prun.cost  manuring.time  manuring.cost
  4   1  prun.time  prun.cost  manuring.time  manuring.cost
  4   2  prun.time  

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            22            22            17            17 
      pd.time       pd.cost 
            0             0 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost 
            2             3             4             5 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1             1             1       1
prun.cost          1         1  

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper")

## Train and test data

Split the dataset into train and test sets. After a model has been processed by using the training set, we test the model by making predictions against the test set. 

Because the data in the testing set already contains known values for the attribute that we want to predict, it is easy to determine whether the model's guesses are correct.

In this case, the datasets from May 2014 until December 2016 is considered for training dataset. The test dataset is from January 2017 until December 2017. 


In [10]:
train = df2[df2$year<=2016,]
test = df2[df2$year==2017,]
dim(train)
dim(test)

[1] 32 11

[1] 12 11

## Regression

Fit the data using multiple regression for all variables. 

In [11]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=train)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-39.550 -23.876  -2.257  17.664  71.865 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)   
(Intercept)    1.624e+02  5.390e+01   3.012  0.00621 **
rainfall      -3.761e-03  9.927e-03  -0.379  0.70830   
foliar        -5.484e+00  1.811e+00  -3.029  0.00597 **
prun.time     -6.510e+00  1.111e+01  -0.586  0.56375   
prun.cost      8.904e-03  6.858e-03   1.298  0.20706   
manuring.time -1.912e+01  1.022e+01  -1.871  0.07416 . 
manuring.cost  2.252e-03  8.315e-04   2.708  0.01254 * 
pd.time       -7.419e+00  8.347e+00  -0.889  0.38327   
pd.cost        2.767e-03  7.036e-03   0.393  0.69777   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 30.68 on 23 degrees of freedom
Multiple R-squared:  0.471,	Adjusted R-squared:  0.2871 
F-statis

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [12]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,1.831770,1.831770,0.00194666,0.965188796
foliar,1,9843.824433,9843.824433,10.46124175,0.003664462
prun.time,1,9.634769,9.634769,0.01023907,0.920277925
prun.cost,1,569.183705,569.183705,0.60488364,0.444647831
manuring.time,1,1775.065544,1775.065544,1.88639994,0.182849678
manuring.cost,1,6275.046962,6275.046962,6.66862597,0.016653606
pd.time,1,653.056050,653.056050,0.69401656,0.413369300
pd.cost,1,145.504968,145.504968,0.15463123,0.697769638
Residuals,23,21642.551370,940.980494,NA,NA


### Using stepwise

In [13]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=226.53
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- rainfall       1     135.0 21778 224.73
- pd.cost        1     145.5 21788 224.75
- prun.time      1     322.9 21965 225.01
- pd.time        1     743.5 22386 225.62
<none>                       21643 226.53
- prun.cost      1    1586.1 23229 226.80
- manuring.time  1    3293.0 24936 229.07
- manuring.cost  1    6902.6 28545 233.39
- foliar         1    8631.2 30274 235.27

Step:  AIC=224.73
yield ~ foliar + prun.time + prun.cost + manuring.time + manuring.cost + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- pd.cost        1     172.1 21950 222.99
- prun.time      1     244.2 22022 223.09
- pd.time        1     726.3 22504 223.78
<none>                       21778 224.73
- prun.cost      1    1760.4 23538 225.22
- manuring.time  1    3309.8 25087 227.26
- manuring.cost  1    7262.4 29040 231.94
-


Call:
lm(formula = yield ~ foliar + prun.cost + manuring.time + manuring.cost, 
    data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-34.475 -24.162  -4.838  17.244  69.476 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.241e+02  1.855e+01   6.687 3.54e-07 ***
foliar        -5.288e+00  1.571e+00  -3.365  0.00231 ** 
prun.cost      6.101e-03  4.310e-03   1.416  0.16834    
manuring.time -1.727e+01  9.338e+00  -1.849  0.07538 .  
manuring.cost  2.159e-03  7.604e-04   2.839  0.00850 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.92 on 27 degrees of freedom
Multiple R-squared:  0.4479,	Adjusted R-squared:  0.3661 
F-statistic: 5.477 on 4 and 27 DF,  p-value: 0.002317


       foliar     prun.cost manuring.time manuring.cost 
     1.039071      1.056044      1.038986      1.059288 



Call:
lm(formula = yield ~ foliar + prun.cost + manuring.time + manuring.cost, 
    data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-34.475 -24.162  -4.838  17.244  69.476 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.241e+02  1.855e+01   6.687 3.54e-07 ***
foliar        -5.288e+00  1.571e+00  -3.365  0.00231 ** 
prun.cost      6.101e-03  4.310e-03   1.416  0.16834    
manuring.time -1.727e+01  9.338e+00  -1.849  0.07538 .  
manuring.cost  2.159e-03  7.604e-04   2.839  0.00850 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.92 on 27 degrees of freedom
Multiple R-squared:  0.4479,	Adjusted R-squared:  0.3661 
F-statistic: 5.477 on 4 and 27 DF,  p-value: 0.002317


## Prediction

The performance of the model is evaluated by comparing the estimated value with the actual value. In this case, the test dataset (2017 datasets) is used to examine the performance of model with significant variables. 

Construct the table and graph that represents the estimated and observed value. 

In [14]:
estimate=predict(selectedMod,newdata=test)
compare=data.frame(test$yield,estimate)
names(compare)=c("Actual","Estimate")
compare

plot(compare$Actual,type="b",col="blue",xlab="Actual",ylab="Predicted",pch=19)
lines(fitted(selectedMod),col="red",type="b",pch=18)
legend("topleft",legend=c("Actual","Predicted"),col=c("blue","red"),lty=1,cex=0.8)

,Actual,Estimate
,<dbl>,<dbl>
33,100.269,80.88821
34,99.035,61.67827
35,112.368,57.03809
36,102.182,45.93854
37,121.640,150.38612
38,72.102,110.65413
39,90.687,111.70790
40,131.546,126.70036
41,84.208,113.28209


## Root mean square error (RMSE)

Calculate the root mean square error (RMSE) of the model. Extract the coefficient of determination (  ) to know how close the data to the fitted regression line. 

In [15]:
rmse=rmse(test$yield,estimate )
fit=c(RMSE = rmse, R2=summary(k1)$r.squared)
fit

RMSE         R2 
33.6732197  0.4710453